In [117]:
!nvidia-smi

Thu Jul  3 21:44:54 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.230.02             Driver Version: 535.230.02   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          On  | 00000000:17:00.0 Off |                    0 |
| N/A   37C    P0              55W / 400W |      4MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [4]:
%pwd

'c:\\Users\\Reeha Parkar\\OneDrive\\Desktop\\Workspace\\KCL\\Semester 2\\Project\\multimodal-emotion-recognition\\CARAT\\dataloaders'

In [5]:
%cd ..

c:\Users\Reeha Parkar\OneDrive\Desktop\Workspace\KCL\Semester 2\Project\multimodal-emotion-recognition\CARAT


In [ ]:
%cd cmu-mosei-experiments/CMU-MultimodalSDK-Tutorials/

In [6]:
import os
import sys
import torch

In [ ]:
data_path = '../CARAT/data/cmu_mosei_unaligned_ree.pt'
print(f"Data file exists: {os.path.exists(data_path)}")

if os.path.exists(data_path):
    print(f"File size: {os.path.getsize(data_path) / (1024*1024):.2f} MB")

try:
    load_data = torch.load(data_path, map_location='cpu')
    print("Dataset loaded successfully!")

except Exception as e:
    print(f"Error loading dataset: {e}")
    sys.exit(1)

Data file exists: True
File size: 9123.32 MB


In [123]:
print(f"Top-level keys: {list(load_data.keys())}")
if 'train' in load_data:
    print(f"Train data keys: {list(load_data['train'].keys())}")

Top-level keys: ['train', 'val', 'test']
Train data keys: ['src-text', 'src-audio', 'src-visual', 'tgt']


In [124]:
expected_keys = ['src-text', 'src-visual', 'src-audio', 'tgt']

In [126]:
for split in ['train', 'val', 'test']:
        if split in load_data:
            split_data = load_data[split]
            for key in expected_keys:
                if key not in split_data:
                    print(f"Missing key '{key}' in split '{split}'")
                else:
                    print(f"Key '{key}' found in split '{split}' with {len(split_data[key])} items")
            print(f"-"*40)
                



Key 'src-text' found in split 'train' with 16322 items
Key 'src-visual' found in split 'train' with 16322 items
Key 'src-audio' found in split 'train' with 16322 items
Key 'tgt' found in split 'train' with 16322 items
----------------------------------------
Key 'src-text' found in split 'val' with 1871 items
Key 'src-visual' found in split 'val' with 1871 items
Key 'src-audio' found in split 'val' with 1871 items
Key 'tgt' found in split 'val' with 1871 items
----------------------------------------
Key 'src-text' found in split 'test' with 4659 items
Key 'src-visual' found in split 'test' with 4659 items
Key 'src-audio' found in split 'test' with 4659 items
Key 'tgt' found in split 'test' with 4659 items
----------------------------------------


In [127]:
print("\n--------- Non-Aligned Data -------------")
# Check the structure of non-aligned data and first 5 key value entries
print(f"Train segments: {len(load_data['train']['src-text'])}")
print(f"Validation segments: {len(load_data['val']['src-text'])}")
print(f"Test segments: {len(load_data['test']['src-text'])}")
print(f"-"*40)
print("First 5 segments in train split:") # To show variable and original time_steps intervals
for i, (text, audio, visual, label) in enumerate(zip(
        load_data['train']['src-text'][:3],
        load_data['train']['src-audio'][:3],
        load_data['train']['src-visual'][:3],
        load_data['train']['tgt'][:3])):
    print(f"\nSegment {i+1}:")
    print(f"  Text shape: {text.shape}")
    print(f"  Audio shape: {audio.shape}")
    print(f"  Visual shape: {visual.shape}")
    print(f"  Label shape: {label.shape}")


--------- Non-Aligned Data -------------
Train segments: 16322
Validation segments: 1871
Test segments: 4659
----------------------------------------
First 5 segments in train split:

Segment 1:
  Text shape: (55, 300)
  Audio shape: (1781, 74)
  Visual shape: (535, 35)
  Label shape: (6,)

Segment 2:
  Text shape: (22, 300)
  Audio shape: (539, 74)
  Visual shape: (162, 35)
  Label shape: (6,)

Segment 3:
  Text shape: (23, 300)
  Audio shape: (922, 74)
  Visual shape: (277, 35)
  Label shape: (6,)


In [130]:
print("Label format check:")
print(f"First segment label: {load_data['train']['tgt'][0]}")
print(f"Label shape: {load_data['train']['tgt'][0].shape}")
print(f"Label dtype: {load_data['train']['tgt'][0].dtype}")

print("\nFirst 5 segments with labels:")
emotion_names = ['happy', 'sad', 'anger', 'surprise', 'disgust', 'fear']

for i in range(3):
    label = load_data['train']['tgt'][-i]
    print(f"\nSegment {i+1}:")
    print(f"  Raw label: {label}")
    print(f"  Label shape: {label.shape}")
    
    # Show which emotions are present
    present_emotions = []
    for j, emotion_val in enumerate(label):
        if emotion_val > 0:  # Assuming binary labels or thresholded values
            present_emotions.append(f"{emotion_names[j]}({emotion_val:.2f})")
    
    if present_emotions:
        print(f"  Present emotions: {', '.join(present_emotions)}")
    else:
        print(f"  No emotions detected (all zeros)")

Label format check:
First segment label: [0. 0. 0. 0. 0. 0.]
Label shape: (6,)
Label dtype: float32

First 5 segments with labels:

Segment 1:
  Raw label: [0. 0. 0. 0. 0. 0.]
  Label shape: (6,)
  No emotions detected (all zeros)

Segment 2:
  Raw label: [0. 0. 0. 0. 0. 0.]
  Label shape: (6,)
  No emotions detected (all zeros)

Segment 3:
  Raw label: [0. 0. 0. 0. 0. 0.]
  Label shape: (6,)
  No emotions detected (all zeros)
